<a href="https://colab.research.google.com/github/eagomez2/mir_basics/blob/main/eagomez_assignment01_exemplary_mir_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIR assignment 01 - Basic flow in an exemplary MIR classification research task 

## Instructions
In this assignment you should understand and reproduce the instrument classification experiment presented in [this jupyter notebook](https://github.com/MTG/MIRCourse/blob/master/notebooks/Lecture1_IntroWithACaseStudy.ipynb), and then adapt the notebook to another instrument classification task, one using the [Mridangam Stroke dataset](https://zenodo.org/record/4068196). The goal is to automatically recognize the 10 different stroke classes or the 6 different tonic classes present in the dataset (you choose), trying to obtain the best possible accuracy. For both tasks you should use the whole dataset.  

The goal of the assignment is to understand and be able to implement all the steps involved in a typical audio-based classification problem. Thus, understanding all the steps involved in designing the experiment and making the best choices for a given task. You should briefly explain within the submitted notebook, all the decisions taken in every step of the experiment.  

You can optionally add a step of feature selection. Feature selection is the removal of redundancies in features, lowering the dimensionality of the feature space. Effective feature selection can improve performance of a classifier and would lower memory and computation requirements.

- Increase the number of classes to 8 (either use more classes from UIOWA:MIS dataset or continue with the Mridangam dataset)

- Add a feature selection step in the process (most critical part)

- Observe and report its contribution, discuss your observations

## About the structure of this notebook

The author has chosen to implement the necessary code for the following tasks:
1. Tonic classification of the Mridangam Stroke dataset
2. Stroke classification of the Mridangam Stroke dataset

Additionally, some techniques for **feature selection** and **class imbalance** are addressed to evaluate their impact in the aforementioned tasks.

# Installing Essentia  
Essentia is an open-source library and tools for audio and music analysis, description and synthesis whose documentation can be found [here](https://essentia.upf.edu/).

In [2]:
# Installs Essentia if it's not installed yet
import importlib.util
if importlib.util.find_spec('essentia') is None:
  !pip install essentia

# Imports  
Some other dependencies that are going to be used for the different sections of this notebook.

In [4]:
import os, sys, shutil, zipfile, requests, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import essentia.standard as ess
import seaborn as sns
from tqdm.notebook import tqdm # helps to create progress bars during loops
from pathlib import Path
from IPython.display import display, Markdown

# Dataset acquisition  
The code in the following sections downloads and uncompresses the Mridangam Stroke dataset, locating it inside the corresponding folder for reading and further processing of the audio files.  

The dataset is comprised of 6 tonic classes and 10 stroke classes named under the following conventions:

> StrokeName_Tonic_InstanceNumber.wav  
> Tonic = {B, C, Csh, D, Dsh, E}  
> StrokeName = {Bheem, Cha, Dheem, Dhin, Num, Ta, Tha, Tham, Thi, Thom}

Additional description about the dataset can be found [here](https://zenodo.org/record/4068196#.YBINk3dKjQg).

## Download helper function  
The following function is created in order to download the dataset appropriately.

In [6]:
def download_dataset(url=None, filename=None, dir='data', chunk_size=1024, unzip=True, delete_original=True, delete_postflight=['__MACOSX']):
  """ Downloads the dataset to a local folder, unzip it and get rids of
      unnecessary content if necessary.

      Args:
          url (str):                Link of the downloadable dataset.
          filename (str):           Name to be given to the dataset file.
          dir (str):                Folder where the dataset is stored.
          chunk_size (int):         Chunk size for the download streaming.
          unzip (bool):             Unzip after download is completed.
          delete_original (bool):   If True, deletes original file after it is uncompressed.
          delete_postflight (list): List of strings containing folders to be deleted after
              the file is downloaded and unziped.
  """
  r = requests.get(url, stream=True)
  total_size = int(r.headers['content-length'])
  filepath = os.path.join(dir, filename)

  # Make folder if it does not exist
  if not os.path.exists(dir):
    os.mkdir(dir)

  # Download the content
  with open(filepath, 'wb') as file:
    for data in tqdm(iterable=r.iter_content(chunk_size=chunk_size), total=(total_size / chunk_size), unit='MB'):
      file.write(data)
    
  print(f'Dataset successfully downloaded to {filepath}')

  # Unzip file
  if unzip:
    print(f'Uncompressing {filename}...')
    zip_extractor = zipfile.ZipFile(filepath, 'r')
    zip_extractor.extractall(dir)
    zip_extractor.close()
    print(f'Uncompressed file saved to {dir}...')

  # Delete original folder
  if delete_original:
    print(f'Deleting original file {filepath}...')
    os.remove(filepath)

  # Delete other unnecessary folders
  if delete_postflight:
    for item in delete_postflight:
      item_path = os.path.join(dir, item)
      if os.path.exists(item_path):
        print(f'Deleting unnecessary item {item_path}')
        shutil.rmtree(item_path)

  print('Operation completed!')

## Dataset related definitions  
Some dataset related constants are stored to be used in later steps.

In [8]:
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
DATA_ROOT = os.path.join(BASE_DIR, 'data')
DATASET_URL = 'https://zenodo.org/record/4068196/files/mridangam_stroke_1.5.zip'
DATASET_NAME = os.path.splitext(os.path.basename(DATASET_URL))[0]
DATASET_ROOT = os.path.join(DATA_ROOT, DATASET_NAME)
DATASET_FS = 44100

# A dict to pattern match file names
TONIC_CLASSES = {
    '-b-':    'b',
    '-c-':    'c',
    '-csh-':  'c#',
    '-d-':    'd',
    '-dsh-':  'd#',
    '-e-':    'e',
}

STROKE_CLASSES = {
    '_bheem-':  'bheem',
    '_cha-':    'cha',
    '_dheem-':  'dheem',
    '_dhin-':   'dhin',
    '_num-':    'num',
    '_ta-':     'ta',
    '_tha-':    'tha',
    '_tham-':   'tham',
    '_thi-':    'thi',
    '_thom-':   'thom',
}

## Download the dataset  
Downloads and uncompresses the dataset at the given location.

In [9]:
download_dataset(url=DATASET_URL, filename=DATASET_URL.split('/')[-1], dir=DATA_ROOT)


Dataset successfully downloaded to /content/data/mridangam_stroke_1.5.zip
Uncompressing mridangam_stroke_1.5.zip...
Uncompressed file saved to /content/data...
Deleting original file /content/data/mridangam_stroke_1.5.zip...
Deleting unnecessary item /content/data/__MACOSX
Operation completed!


True